In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import seaborn as sns

In [3]:
# Read the data
data = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv', index_col='Id')
data_test = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv', index_col='Id')

In [4]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
# Drop columns with many missing values
data.drop(['Alley','PoolQC', 'Fence', 'MiscFeature', 'FireplaceQu'], axis=1, inplace=True)

In [6]:
# Separate target from predictors
y = data.SalePrice
X = data.drop(['SalePrice'], axis=1)
X_test = data_test

In [7]:
# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X,y,train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Select categorical columns with relatively low cardinality
categorical_cols = [cname for cname in X_train_full.columns
                    if X_train_full[cname].nunique()<10 and 
                    X_train_full[cname].dtype=='object']

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns
                 if X_train_full[cname].dtype in ['int64','float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train_mycols = X_train_full[my_cols].copy()
X_valid_mycols = X_valid_full[my_cols].copy()
X_test_mycols = X_test[my_cols].copy()

X_train_mycols.head()
categorical_cols

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [8]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for both numerical and categorical data
preprocessor = ColumnTransformer(
    transformers = [
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
])

In [9]:
X_train_mycols.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,BldgType,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,...,774,0,108,0,0,260,0,0,7,2007
871,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,PosN,Norm,1Fam,...,308,0,0,0,0,0,0,0,8,2009
93,RL,Pave,IR1,HLS,AllPub,Inside,Gtl,Norm,Norm,1Fam,...,432,0,0,44,0,0,0,0,8,2009
818,RL,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Norm,Norm,1Fam,...,857,150,59,0,0,0,0,0,7,2008
303,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,1Fam,...,843,468,81,0,0,0,0,0,1,2006


In [10]:
data_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [11]:
# Define random forest models with different values of n_estimators
model_rf1 = RandomForestRegressor(n_estimators=75, random_state=0)
model_rf2 = RandomForestRegressor(n_estimators=150, random_state=0)
model_rf3 = RandomForestRegressor(n_estimators=225, random_state=0)
model_rf4 = RandomForestRegressor(n_estimators=300, random_state=0)
model_rf5 = RandomForestRegressor(n_estimators=375, random_state=0)

In [12]:
# Define random gradient boost models with different values of n_estimators
model_xgb1 = XGBRegressor(n_estimators=150, learning_rate=0.05, random_state=0)
model_xgb2 = XGBRegressor(n_estimators=400, learning_rate=0.05, random_state=0)
model_xgb3 = XGBRegressor(n_estimators=650, learning_rate=0.05, random_state=0)
model_xgb4 = XGBRegressor(n_estimators=900, learning_rate=0.05, random_state=0)
model_xgb5 = XGBRegressor(n_estimators=1150, learning_rate=0.05, random_state=0)

In [13]:
# Group the models in a list
models = model_rf1,model_rf2,model_rf3,model_rf4,model_rf5,model_xgb1,model_xgb2,model_xgb3,model_xgb4,model_xgb5

In [14]:
# Function for comparing differert models
def score_model(model, X_t=X_train_mycols,X_v=X_valid_mycols,y_t=y_train,y_v=y_valid):
    my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                  ('model', model)
                                 ])
    my_pipeline.fit(X_t, y_t)
    preds = my_pipeline.predict(X_v)
    return mean_absolute_error(y_v, preds)

for i in range(0, len(models)):
    mae = score_model(models[i])
    print("Model " + str(i+1) + " MAE: " + str(mae))

Model 1 MAE: 17807.46771689498
Model 2 MAE: 17428.115867579912
Model 3 MAE: 17415.867442922372
Model 4 MAE: 17430.506700913244
Model 5 MAE: 17394.559324200916
Model 6 MAE: 16571.998983304795
Model 7 MAE: 16342.83965646404
Model 8 MAE: 16346.471372003425
Model 9 MAE: 16374.655046018835
Model 10 MAE: 16389.34532855308


As Model 7 is has the smallest MAE, it us the 7th item in the variable "models"

In [15]:
# Assign the best model as a variable
best_model = model_xgb2

Fitting the best model using a pipeline

In [16]:
# Bundle preprocessing and the defined best model
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('model', best_model)])
# Fit model
my_pipeline.fit(X_train_mycols, y_train)

# Generate prediction
preds = my_pipeline.predict(X_valid_mycols)

# Evaluate the model
MAE_best = mean_absolute_error(y_valid, preds)
print("Best MAE: " + str(MAE_best))


Best MAE: 16342.83965646404


In [17]:
# Generate test predictions
preds_test = my_pipeline.predict(X_test_mycols)

# Create Output format for predictions
output = pd.DataFrame({'Id': data_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)
